In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from flask import Flask, render_template, jsonify, send_from_directory
from sqlalchemy import create_engine, inspect

# ================================
# PARTE 1: Extracción desde jBPM
# ================================

# Parámetros de conexión
JBPM_HOST = "http://localhost:8080"
USERNAME = "wbadmin"
PASSWORD = "wbadmin"
CONTAINER_ID = "Publica_In_Out_1.0.0-SNAPSHOT"

headers = {
    "Accept": "application/json"
}

# Paso 1: Obtener último processInstanceId activo
def get_latest_process_instance():
    url = f"{JBPM_HOST}/kie-server/services/rest/server/queries/processes/instances?status=1&page=0&pageSize=10"
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth(USERNAME, PASSWORD))
    if response.status_code == 200:
        instances = response.json().get("process-instance", [])
        if instances:
            return instances[0].get("process-instance-id")
    return None

# Paso 2: Extraer todos los documentos de cualquier variable
def get_all_documents(process_instance_id):
    url = f"{JBPM_HOST}/kie-server/services/rest/server/containers/{CONTAINER_ID}/processes/instances/{process_instance_id}/variables"
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth(USERNAME, PASSWORD))
    resultados = []

    if response.status_code == 200:
        data = response.json()

        for var_name, var_value in data.items():
            if isinstance(var_value, dict) and "documents" in var_value:
                for doc in var_value["documents"]:
                    doc_data = doc.get("org.jbpm.document.service.impl.DocumentImpl", {})
                    name = doc_data.get("name", "sin_nombre")
                    identifier = doc_data.get("identifier", "sin_id")
                    date = doc_data.get("lastModified", {}).get("java.util.Date", None)
                    value = f"{name}####{identifier}"
                    
                    resultados.append({
                        "processinstanceid": process_instance_id,
                        "value": value,
                        "lastModified": date,
                        "variable": var_name,
                        "identifier": identifier
                    })
    else:
        print(f"Error al obtener variables del proceso {process_instance_id}: {response.status_code}")
        print(response.text)
    
    return resultados

# Ejecutar flujo completo de extracción
process_instance_id = get_latest_process_instance()
if process_instance_id:
    docs = get_all_documents(process_instance_id)
    dfCollect = pd.DataFrame(docs)
    print(dfCollect)
else:
    print("No se encontró ningún proceso activo.")
    dfCollect = pd.DataFrame()  # Evita errores posteriores

# ================================
# PARTE 2: Almacenamiento en PostgreSQL
# ================================

# Configuración de conexión a PostgreSQL
user = 'jbpm'
password = 'jbpm'
host = 'localhost'
port = '5432'
database = 'jbpm'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

app = Flask(__name__)  # Flask declarado, aunque no se usa aquí directamente

# Guardar en base de datos solo si hay datos válidos
if not dfCollect.empty:
    with engine.connect() as conn:
        inspector = inspect(engine)
        tables = inspector.get_table_names()

        if 'tabla_document_collections' not in tables:
            dfCollect.to_sql(
                name='tabla_document_collections',
                con=engine,
                if_exists='replace',
                index=False
            )
            print("Tabla creada e información insertada.")
        else:
            existing_identifiers = pd.read_sql(
                'SELECT identifier FROM tabla_document_collections',
                con=engine
            )['identifier'].astype(str).tolist()

            df_nuevos = dfCollect[~dfCollect['identifier'].astype(str).isin(existing_identifiers)]

            if not df_nuevos.empty:
                df_nuevos.to_sql(
                    name='tabla_document_collections',
                    con=engine,
                    if_exists='append',
                    index=False
                )
                print(f"{len(df_nuevos)} documento(s) insertado(s) exitosamente.")
            else:
                print("No hay nuevos documentos para insertar (identifiers ya existentes).")
else:
    print("No se encontraron documentos para guardar en la base de datos.")

# Verificación final del contenido en la tabla
df_verificacion = pd.read_sql('SELECT * FROM tabla_document_collections', engine)
print("Últimos registros en tabla_document_collections:")
df_verificacion.tail(50)


   processinstanceid                                      value  \
0                 94                    .gitignore####949538214   
1                 94  api_copiar_adjuntos-Copy1.py####767887848   
2                 94         api_copiar_limpiar.py####732052869   

    lastModified         variable identifier  
0  1747262005846   documentosSETN  949538214  
1  1746632552421   documentosSETN  767887848  
2  1747264207023  documentosSRTIC  732052869  
3 documento(s) insertado(s) exitosamente.
Últimos registros en tabla_document_collections:


,processinstanceid,value,lastModified,variable,identifier
0,76,CSD_FUNK671228PH6_20230509_130458.sdg####42227453,1683659102000,documentosSETN,42227453
1,76,CSD_Sucursal_1_FUNK671228PH6_20230509_130451.c...,1684423334000,documentosSETN,451591489
2,76,CSD_Sucursal_1_FUNK671228PH6_20230509_130451.k...,1683659102000,documentosSETN,622413742
3,76,CSD_FUNK671228PH6_20230509_130458.sdg####26811...,1683659102000,documentosSRTIC,268117661
4,76,CSD_Sucursal_1_FUNK671228PH6_20230509_130451.c...,1684423334000,documentosSRTIC,37307117
5,76,CSD_Sucursal_1_FUNK671228PH6_20230509_130451.k...,1683659102000,documentosSRTIC,569366915
6,77,formulario_carga_archivos-taskform (1).frm####...,1745863528002,documentosSETN,571058653
7,77,ambienteAnaconda.rtf####949245523,1745519828070,documentosSETN,949245523
8,87,afirme (1).pdf####951176812,1747153925793,documentosSETN,951176812
9,87,afirme.pdf####453402779,1747153528332,documentosSETN,453402779


In [1]:
from sqlalchemy import create_engine, text
import pandas as pd

# Parámetros de conexión
user = 'jbpm'
password = 'jbpm'
host = 'localhost'
port = '5432'
database = 'jbpm'

# conexión
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Probar conexión y listar tablas
with engine.connect() as connection:
    result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    for row in result:
        # Imprimir usando el índice de la tupla
        #print(row[0])  # row[0] es el valor de la primera columna (table_name)
        print(row)




('booleanexpression',)
('audittaskimpl',)
('bamtasksummary',)
('correlationkeyinfo',)
('casefiledatalog',)
('caseidinfo',)
('caseroleassignmentlog',)
('content',)
('contextmappinginfo',)
('deadline',)
('attachment',)
('delegation_delegates',)
('deploymentstore',)
('email_header',)
('escalation',)
('errorinfo',)
('executionerrorinfo',)
('notification',)
('nodeinstancelog',)
('i18ntext',)
('notification_bas',)
('notification_email_header',)
('notification_recipients',)
('processinstancelog',)
('querydefinitionstore',)
('processinstanceinfo',)
('sessioninfo',)
('requestinfo',)
('taskdef',)
('task',)
('reassignment',)
('peopleassignments_bas',)
('peopleassignments_exclowners',)
('peopleassignments_potowners',)
('peopleassignments_recipients',)
('peopleassignments_stakeholders',)
('reassignment_potentialowners',)
('task_comment',)
('taskevent',)
('taskvariableimpl',)
('variableinstancelog',)
('workiteminfo',)
('organizationalentity',)
('correlationpropertyinfo',)
('eventtypes',)
('documents

In [2]:
collection = pd.read_sql('SELECT * FROM tabla_document_collections', engine)
collection.head(3)

,processinstanceid,value,lastModified,variable,identifier
0,11,geo-3 (5).kml####103825666,1747260380537,documentosSETN,103825666
1,12,Publica (2) - copia.bpmn####197639564,1747262798734,documentosSETN,197639564
2,12,Publica (2).bpmn####524149908,1747262798734,documentosSETN,524149908
3,13,Publica (2) - copia.bpmn####777086027,1747262798734,documentosSETN,777086027
4,13,Publica (2).bpmn####513350129,1747262798734,documentosSETN,513350129
5,13,geo-3 (5).kml####752361240,1747260380537,documentosSETN,752361240
6,15,geo-3 (4).kml####746577378,1747169231280,documentosSETN,746577378
7,15,857608_10_00027_00100_COLF930507HDFNXR03_27772...,1747167132850,documentosSETN,174430747
8,15,857608_10_00027_00100_COLF930507HDFNXR03_27772...,1747167132850,documentosSRTIC,756508079


In [4]:
completa = pd.read_sql('SELECT * FROM x_mi_tabla_completa', engine)
completa.head(3)

,usuario,numero_catastral,value,name,processid,processinstanceid
0,wbadmin,None,org.jbpm.document.service.impl.DocumentCollect...,SolicitudEntregaTerrenosNacionales,Publica_In_Out.Publica,1
1,wbadmin,None,org.jbpm.document.service.impl.DocumentCollect...,Sedatu_RevisonTecnicaeInvestigacionCatastral,Publica_In_Out.Publica,1
2,wbadmin,None,org.jbpm.document.service.impl.DocumentCollect...,SolicitudEntregaTerrenosNacionales,Publica_In_Out.Publica,4
